In [12]:
# First, adjust path to find setup_template
import sys
from pathlib import Path

# Add the project root to the Python path
notebook_dir = Path().absolute()  # Current directory (notebooks folder)
project_root = notebook_dir.parent  # Go up one level to project root
sys.path.append(str(project_root))

# Now import setup_template
from setup_templates import setup_templates
template_dir = setup_templates()

print("Template directory:", template_dir)

# Import required libraries and modules
import os
from pathlib import Path
from typing import List, Dict
from dotenv import load_dotenv
from IPython.display import display, HTML
import pandas as pd
import plotly.graph_objects as go

# Import our template generator modules using absolute imports
from ableton_template_generator.services.template_service import TemplateService
from ableton_template_generator.services.pattern_service import PatternService
from ableton_template_generator.repositories.template_repository import TemplateRepository
from ableton_template_generator.repositories.pattern_repository import PatternRepository
from ableton_template_generator.models.template import Template
from ableton_template_generator.models.group import Group
from ableton_template_generator.models.track import Track, TrackType, ColorCode
from ableton_template_generator.models.timeline import TimelineMarker
from ableton_template_generator.models.midi_pattern import MidiPattern, SessionClip
from ableton_template_generator.config.ai_config import AIConfig

# Load environment variables
load_dotenv()

Templates created in: /Users/luisvalencia/Desktop/PythonPersonalProjects/AbletonTemplateGenerator-1/src/ableton_template_generator/templates
Template directory: /Users/luisvalencia/Desktop/PythonPersonalProjects/AbletonTemplateGenerator-1/src/ableton_template_generator/templates


False

In [18]:
def initialize_services():
    # Get project root directory
    project_root = Path().absolute().parent  # Go up one level from notebooks
    templates_dir = project_root / "src" / "ableton_template_generator" / "templates"
    
    # Initialize repositories with correct paths
    template_repo = TemplateRepository(str(templates_dir))
    pattern_repo = PatternRepository()
    
    template_service = TemplateService(template_repo)
    pattern_service = PatternService(pattern_repo)
    
    return template_service, pattern_service

def generate_template(genres: List[str], with_patterns: bool = True):
    print(f"Generating template for genres: {', '.join(genres)}")
    
    # Initialize services
    template_service, pattern_service = initialize_services()
    
    # Generate template
    template = template_service.create_template(genres)
    
    # Generate patterns if requested
    all_patterns = []
    if with_patterns:
        for group in template.groups:
            for track in group.tracks:
                patterns = pattern_service.get_patterns_for_track(template.genre, track.name)
                if patterns:
                    all_patterns.extend(patterns)
    
    return template, all_patterns if with_patterns else None

import gzip
import xml.etree.ElementTree as ET
from xml.dom import minidom
from pathlib import Path
from typing import List, Dict, Optional
import uuid

def create_ableton_xml(template: Template, patterns: Optional[List[SessionClip]] = None) -> ET.Element:
    """Create Ableton Live XML structure"""
    # Create root element with Ableton Live 11 attributes
    root = ET.Element('Ableton', {
        'MajorVersion': '5',
        'MinorVersion': '10.0_377',
        'SchemaChangeCount': '3',
        'Creator': 'Ableton Live 11.0.12',
        'Revision': '570160e452e6e8ec882f7b3e365122a4f3af9abd'
    })

    # Add LiveSet element
    live_set = ET.SubElement(root, 'LiveSet')

    # Add Tracks container
    tracks = ET.SubElement(live_set, 'Tracks')

    # Process each group
    for group in template.groups:
        # Create group track
        group_track = ET.SubElement(tracks, 'GroupTrack')
        group_track.set('Id', str(uuid.uuid4()))
        group_track.set('Name', group.name)
        
        # Add group color
        color = ET.SubElement(group_track, 'ColorIndex')
        color.text = str(_convert_color_to_live_index(group.color))
        
        # Add group tracks container
        group_tracks = ET.SubElement(group_track, 'Tracks')
        
        # Process each track in group
        for track in group.tracks:
            track_element = ET.SubElement(group_tracks, 
                                        'MidiTrack' if track.type == TrackType.MIDI else 'AudioTrack')
            track_element.set('Id', str(uuid.uuid4()))
            track_element.set('Name', track.name)
            
            # Add track color
            track_color = ET.SubElement(track_element, 'ColorIndex')
            track_color.text = str(_convert_color_to_live_index(track.color))
            
            # Add device chain
            devices = ET.SubElement(track_element, 'DeviceChain')
            
            # Add MIDI patterns if available
            if patterns:
                track_patterns = [p for p in patterns 
                                if p.name.startswith(f"{group.name} - {track.name}")]
                if track_patterns:
                    _add_patterns_to_track(devices, track_patterns)

    # Add master track
    master = ET.SubElement(tracks, 'MasterTrack')
    master.set('Id', str(uuid.uuid4()))
    master.set('Name', 'Master')
    
    # Add tempo and other global settings
    tempo = ET.SubElement(live_set, 'MasterTrack')
    tempo.set('Value', str(template.default_tempo))
    
    # Add timeline markers if any
    if template.timeline_markers:
        locators = ET.SubElement(live_set, 'Locators')
        for marker in template.timeline_markers:
            locator = ET.SubElement(locators, 'Locator')
            locator.set('Id', str(uuid.uuid4()))
            locator.set('Name', marker.name)
            locator.set('Time', str(marker.position_bars * 4))  # Convert bars to beats
            locator.set('Duration', str(marker.duration_bars * 4))
            locator.set('Label', marker.description)

    return root

def _convert_color_to_live_index(color: ColorCode) -> int:
    """Convert our color codes to Ableton Live color indices"""
    color_map = {
        ColorCode.BLUE: 5,    # Blue
        ColorCode.YELLOW: 1,  # Yellow
        ColorCode.RED: 3,     # Red
        ColorCode.GREEN: 2,   # Green
        ColorCode.PURPLE: 7,  # Purple
        ColorCode.ORANGE: 4   # Orange
    }
    return color_map.get(color, 0)

def _add_patterns_to_track(devices: ET.Element, patterns: List[SessionClip]):
    """Add MIDI patterns to track's device chain"""
    clip_slots = ET.SubElement(devices, 'ClipSlots')
    
    for clip in patterns:
        slot = ET.SubElement(clip_slots, 'ClipSlot')
        slot.set('Id', str(uuid.uuid4()))
        slot.set('Time', str(clip.pattern.length_bars * 4))
        
        clip_element = ET.SubElement(slot, 'MidiClip')
        clip_element.set('Name', clip.name)
        
        notes = ET.SubElement(clip_element, 'Notes')
        for note in clip.pattern.notes:
            note_element = ET.SubElement(notes, 'Note')
            note_element.set('Time', str(note.position))
            note_element.set('Duration', str(note.duration))
            note_element.set('Velocity', str(note.velocity))
            note_element.set('Pitch', str(note.pitch))

def export_to_ableton(template: Template, 
                     patterns: Optional[List[SessionClip]] = None,
                     output_path: str = "template.als"):
    """Export template to Ableton Live format (.als)"""
    # Create XML structure
    root = create_ableton_xml(template, patterns)
    
    # Convert to pretty XML string
    xml_str = minidom.parseString(ET.tostring(root)).toprettyxml(indent="  ")
    
    # Convert output path to Path object
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Save uncompressed XML for debugging (optional)
    xml_path = output_path.with_suffix('.xml')
    with open(xml_path, 'w') as f:
        f.write(xml_str)
    print(f"Saved uncompressed XML to: {xml_path}")
    
    # Compress XML to .als format
    with gzip.open(output_path, 'wb') as f:
        f.write(xml_str.encode('utf-8'))
    print(f"Saved Ableton Live template to: {output_path}")

    return output_path

In [19]:
# Generate template for Cumbia and Nu Disco
genres = ['cumbia', 'nudisco']
template, patterns = generate_template(genres)

# Export to Ableton format
export_to_ableton(
    template=template,
    patterns=patterns,
    output_path="output/template.als"
)

Generating template for genres: cumbia, nudisco
Template directory: /Users/luisvalencia/Desktop/PythonPersonalProjects/AbletonTemplateGenerator-1/src/ableton_template_generator/templates
Looking for template at: /Users/luisvalencia/Desktop/PythonPersonalProjects/AbletonTemplateGenerator-1/src/ableton_template_generator/templates/cumbia.json
Looking for template at: /Users/luisvalencia/Desktop/PythonPersonalProjects/AbletonTemplateGenerator-1/src/ableton_template_generator/templates/nudisco.json
Saved uncompressed XML to: output/template.xml


IsADirectoryError: [Errno 21] Is a directory: 'output/template.als'